In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
def Sigmoid(x):
  return 1/(1 + np.exp(-x)) #np.inf*x for step function

def D_Sigmoid(x):
  return Sigmoid(x)*(1-Sigmoid(x))

def Step(x):
  return int(np.any(x) > 0)

def GetLabel(x, y, a, b, r):
  return int((x-a)**2 + (y-b)**2 < r**2) #1 = +, 0 = -

#y_hat = true labels
#y = predicted labels
#Mean Square Error
def Loss(y, y_hat):
  return np.mean((y_hat - y)**2)/2

def D_Loss(y, y_hat):
  return -np.mean(y_hat - y)

In [ ]:
#Generate train data
dataset_x_train = np.random.uniform(low=0, high=1, size=100)
dataset_y_train = np.random.uniform(low=0, high=1, size=100)
labels_train = [GetLabel(x, y, 0.5, 0.6, 0.4) for x,y in zip(dataset_x_train, dataset_y_train)]

#Generate train data
dataset_x_test = np.random.uniform(low=0, high=1, size=100)
dataset_y_test = np.random.uniform(low=0, high=1, size=100)
labels_test = [GetLabel(x, y, 0.5, 0.6, 0.4) for x,y in zip(dataset_x_test, dataset_y_test)]

eta = 0.01

In [ ]:
class Node():
  def __init__(self, num_weights):
    self.weights = np.random.normal(0, 1, size=num_weights+1)

  def __call__(self, x):
    x = np.append(x, 1) #add bias term
    return Sigmoid(np.sum(x * self.weights))

  def backprop(self, x, curr_gradient):
    x = np.append(x, 1) #add bias term
    curr_gradient = curr_gradient*D_Sigmoid(x)
    self.weights = self.weights - eta*curr_gradient*x
    return curr_gradient*self.weights #return gradient to be passed back

In [ ]:
#initialize network

#input_layer = [Node(2) for i in range(2)]

hidden_layer = [Node(2) for i in range(10)]

output_layer = [Node(10) for i in range(1)]

In [ ]:
#Training Loop

epochs = 1000
train_loss_list = []
test_loss_list = []
mean_train_loss_list = []
mean_test_loss_list = []

for i in tqdm(range(epochs)):
  #Training
  epoch_train_loss_list = []
  for j in range(100):
    #Get current datapoint
    curr_x = dataset_x_train[j]
    curr_y = dataset_y_train[j]
    curr_label = labels_train[j]

    #Forward
    #input_out = []
    #for node in input_layer:
    #  input_out.append(node([curr_x, curr_y]))


    hidden_out = []
    for node in hidden_layer:
      hidden_out.append(node([curr_x, curr_y]))

    output_out = []
    for node in output_layer:
      output_out.append(node(hidden_out))

    #Loss
    curr_loss = Loss(output_out[0], curr_label)
    train_loss_list.append(curr_loss)
    epoch_train_loss_list.append(curr_loss)
    d_loss = D_Loss(output_out[0], curr_label)

    #Backward
    output_grad = []
    for node in output_layer:
      output_grad.append(node.backprop(hidden_out, d_loss))

    hidden_grad = []
    for k, node in enumerate(hidden_layer):
      hidden_grad.append(node.backprop([curr_x, curr_y], output_grad[0][k]))

    #input_grad = []
    #for node in input_layer:
    #  for grad in hidden_grad:
    #    node.backprop([curr_x, curr_y], grad/len(hidden_grad)) #Mean backprop
        #input_grad.append()
  mean_train_loss_list.append(np.mean(epoch_train_loss_list))

  #Testing
  epoch_test_loss_list = []
  for j in range(100):
    #Get current datapoint
    curr_x = dataset_x_test[j]
    curr_y = dataset_y_test[j]
    curr_label = labels_test[j]

    #Forward
    #input_out = []
    #for node in input_layer:
    #  input_out.append(node([curr_x, curr_y]))


    hidden_out = []
    for node in hidden_layer:
      hidden_out.append(node([curr_x, curr_y]))

    output_out = []
    for node in output_layer:
      output_out.append(node(hidden_out))

    #Loss
    curr_loss = Loss(output_out[0], curr_label)
    test_loss_list.append(curr_loss)
    epoch_test_loss_list.append(curr_loss)

  mean_test_loss_list.append(np.mean(epoch_test_loss_list))


In [ ]:
plt.title("Train Loss curve")
plt.xlabel("Epochs")
plt.ylabel("Train Loss")
plt.plot(mean_train_loss_list)

In [ ]:
plt.title("Test Loss curve")
plt.xlabel("Epochs")
plt.ylabel("Test Loss")
plt.plot(mean_test_loss_list)

In [ ]:
xs = np.linspace(0,1,100)
ys = np.linspace(0,1,100)

X, Y = np.meshgrid(xs, ys)

fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.view_init(30, 30)
plt.xlabel('x')
plt.ylabel('y')
for i, node in enumerate(hidden_layer):
  if i<2: continue
  z = Sigmoid(node.weights[0] * X + node.weights[1] * Y + node.weights[2])
  ax.plot_surface(X, Y, z)
  break